# Ant Colony Optimization Problem
#### by Vico Wietstock, Jakob Litsch, Alexander Palatnik, Paul Ludwig Branzk and Friedrich Heitzer

In [1]:
import numpy as np

### Input, Initialization (Friedrich Heitzer)

In [2]:
eingabe = input("Welche benchmark soll laufen? ")

#Reads the tsp file and creates a nested list out of it
liste = []
with open(eingabe + ".tsp") as f:
    for line in f:
        linelist = line.split(" ")
        linelist = linelist[:-1]
        linelist = list(map(int, linelist))
        liste.append(linelist)

# numpy array for distances between cities
distances = np.array(liste)

# number of cities
citycount = len(liste[0])

# Initialization of the pheromones array
pheromones = np.ones((citycount, citycount))
for i in range(citycount):
    pheromones[i][i] = 0

antcount_values = [10, 50, 100]

evap_values = [0.9, 0.7, 0.5]

intens_values = [0.1, 0.3, 0.5]

alpha_values = [0, 1, 2]

beta_values = [0, 1, 2]


Welche benchmark soll laufen? 01manhattan


### Construct Solution (Alexander Palatnik, Jakob Litsch)

In [21]:
def constructsolution(pval, dval, citycount, a, b, antcount):
    solution = np.zeros((antcount,citycount-1), dtype = int)
    cities = np.zeros((citycount,), dtype = float)
    for ant in range(antcount):
        pos = 0
        chosencities = np.zeros((citycount+1,), dtype = int)
        chosencities[0] = pos
        for city in range(citycount):
            if(city in chosencities):
                cities[city] = 0
            else:
                cities[city] = computeprob(pos, city, pval, dval, a, b, chosencities)
        for i in range(citycount-1):
            pos = np.random.choice(len(cities), 1, p = cities)
            chosencities[i+1] = pos
            solution[ant,i] = pos
            for city in range(citycount):
                if(city in chosencities):
                    cities[city] = 0
                else:
                    cities[city] = computeprob(pos, city, pval, dval, a, b, chosencities)
    return solution

In [4]:
def computeprob(pos, city, pval, dval, a, b, chosencities):
    p = (1/computeN(pos, pval, dval, a, b, chosencities))* ((pval[pos, city]**a)/(dval[pos, city]**b))
    return p

In [5]:
def computeN(pos, pval, dval, a, b, chosencities):
    N = 0
    for index in range(len(dval)):
        if index not in chosencities:
            N += pval[pos, index]**a/dval[pos, index]**b
            
    return N

### Evaluation, Evaporization, Intensification (Vico Wietstock, Friedrich Heitzer)

In [6]:
# returns distance of the given solution/ant 
def get_distance(distances, ant):
    
    distance = 0        
    # iterates through cities in the ant and sums up distances 
    for city, nextcity in zip(ant[:-1], ant[1:]):
        distance += distances[city][nextcity]
        
    return distance

In [7]:
# returns the best solution/ant
def get_best(ants):
    values = []
    # Interates through ant population
    for ant in ants:
        distance = 0
        
        # double iterates through the cities in the seperate solutions aka ants
        for city, nextcity in zip(ant[:-1], ant[1:]):
            distance += distances[city][nextcity]
        values.append(distance)

    # returns the best solution aka ant
    return ants[values.index(min(values))]

In [16]:
# Evaporization 
def evaporize(pheromones, evaporize_val):
    # evaporize all pheromone values
    pheromones *= evaporize_val
    
    return pheromones

In [17]:
# Intensification
def intensify(pheromones, intens_val, ants):
    # get best solution 
    best_ant = get_best(ants)
    
    # intensify values from the best solution
    for city, nextcity in zip(best_ant[:-1], best_ant[1:]):
        pheromones[city][nextcity] += intens_val
        
    return pheromones, best_ant

### Main (Ludwig Branzk, Vico Wietstock)

In [24]:
def main(distances, pheromones, citycount, antcount, evaporize_val, intens_val, alpha, beta):
    
    population = constructsolution(pheromones, distances, citycount, alpha, beta, antcount)
    pheromones = evaporize(pheromones, evaporize_val)
    pheromones, best_solution = intensify(pheromones, intens_val, population)
    
    best_distance = get_distance(distances, best_solution)
    
    counter = 0
    
    while counter < 5:
        population = constructsolution(pheromones, distances, citycount, alpha, beta, antcount)
        pheromones = evaporize(pheromones, evaporize_val)
        pheromones, best_ant = intensify(pheromones, intens_val, population)
        
        distance = get_distance(distances, best_ant)
        
        if distance <= best_distance:
            best_solution = best_ant
            best_distance = distance
            counter = 0
        else:
            counter += 1
        print(counter)
    return best_distance

In [25]:
antcount = 2
evaporize_val = 0.5
intens_val = 0.5
alpha = 1
beta = 1
print(main(distances, pheromones, citycount, antcount, evaporize_val, intens_val, alpha, beta))

0
0
0
0
1
0
0
0
0
0
1
2
3
4
5
10902
